In [1]:
import numpy
import pandas
import tensorflow as tf
 
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn import metrics

models =  tf.keras.models
layers = tf.keras.layers 
scikit_learn = tf.keras.wrappers.scikit_learn



/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
Sequential = models.Sequential
Dense = layers.Dense
KerasClassifier = scikit_learn.KerasClassifier
to_categorical = tf.keras.utils.to_categorical
SGD = tf.keras.optimizers.SGD

In [3]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [4]:

def load_data():
    dataframe = pandas.read_csv("sonar.csv", header=None)
    dataset = dataframe.values
    # split into input (X) and output (Y) variables
    X = dataset[:,0:60].astype(float)
    Y = dataset[:,60]
    le = LabelEncoder()
    le.fit(Y)
    encoded_Y = le.transform(Y)
    #X_train, X_test, Y_train, Y_test = train_test_split(X, encoded_Y, test_size=.05)
    
    #return (X_train, Y_train, X_test, Y_test)
    return (X, encoded_Y, [], [])

(X_train, Y_train, X_test, Y_test) = load_data()

In [5]:
def create_baseline():
     
    model = Sequential()    
    model.add(layers.Dense(60, activation='relu', input_shape=(60, )))
    #model.add(layers.Dropout(.2))
    model.add(layers.Dense(30, activation='relu'))
    #model.add(layers.Dropout(.2))
    model.add(layers.Dense(1, activation='sigmoid'))
    sgd = SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=False)
    model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [6]:
def do_normalize(epochs=100): 
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=epochs, batch_size=5, verbose=0)))
    pipeline = Pipeline(estimators)
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
    results = cross_val_score(pipeline, X_train, Y_train, cv=kfold)
    print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    #predicted = cross_val_predict(pipeline, X_test, Y_test, cv=10)
    #print("Test Accuracy: %.2f%%" % (metrics.accuracy_score(Y_test, predicted)*100))

In [7]:
#(X_train, Y_train, X_test, Y_test) = load_data()
do_normalize(100)


Standardized: 85.97% (8.79%)


In [8]:
def create_dropout_visible():
    model = Sequential()    
    model.add(layers.Dropout(0.2, input_shape=(60, )))
    model.add(layers.Dense(60, activation='relu'))
    model.add(layers.Dense(30, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    sgd = SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=False)
    model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [9]:
def do_normalize_dropout_visible(epochs=100):
    numpy.random.seed(seed)
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('mlp', KerasClassifier(build_fn=create_dropout_visible, epochs=epochs, batch_size=5, verbose=0)))
    pipeline = Pipeline(estimators)
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
    results = cross_val_score(pipeline, X_train, Y_train, cv=kfold)
    print("Visible: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    #predicted = cross_val_predict(pipeline, X_test, Y_test, cv=10)
    #print("Test Accuracy: %.2f%%" % (metrics.accuracy_score(Y_test, predicted)*100))

In [10]:
#(X_train, Y_train, X_test, Y_test) = load_data()
do_normalize_dropout_visible(100)

Visible: 85.13% (6.46%)


In [11]:
max_norm = tf.keras.constraints.max_norm
def create_dropout_visible2():
    model = Sequential()    
    model.add(layers.Dropout(0.2, input_shape=(60, )))
    model.add(layers.Dense(60, activation='relu', kernel_constraint=max_norm(3.)))
    model.add(layers.Dense(30, activation='relu', kernel_constraint=max_norm(3.)))
    model.add(layers.Dense(1, activation='sigmoid'))
    sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False)
    model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [12]:
def do_normalize_dropout_visible2(epochs=100):
    numpy.random.seed(seed)
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('mlp', KerasClassifier(build_fn=create_dropout_visible2, epochs=epochs, batch_size=5, verbose=0)))
    pipeline = Pipeline(estimators)
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
    results = cross_val_score(pipeline, X_train, Y_train, cv=kfold)
    print("Visible 2: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100)) 

In [13]:
do_normalize_dropout_visible2(100)

Visible 2: 49.52% (3.57%)


In [14]:
def create_dropout_hidden():
    model = Sequential() 
    model.add(layers.Dense(60, activation='relu', kernel_constraint=max_norm(3.), input_shape=(60, )))
    model.add(layers.Dropout(.2))
    model.add(layers.Dense(30, activation='relu', kernel_constraint=max_norm(3.)))
    model.add(layers.Dropout(.2))
    model.add(layers.Dense(1, activation='sigmoid'))
    sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False)
    model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [15]:
def do_normalize_dropout_hidden(epochs=100):
    numpy.random.seed(seed)
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('mlp', KerasClassifier(build_fn=create_dropout_hidden, epochs=epochs, batch_size=5, verbose=0)))
    pipeline = Pipeline(estimators)
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
    results = cross_val_score(pipeline, X_train, Y_train, cv=kfold)
    print("Hidden: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100)) 

In [16]:
do_normalize_dropout_hidden(100)

Hidden: 49.52% (3.57%)


In [17]:
dropout = .2
def create_dropout_multiple_dropout():
    model = Sequential() 
    model.add(layers.Dense(60, activation='relu', kernel_constraint=max_norm(3.), input_shape=(60, )))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(30, activation='relu', kernel_constraint=max_norm(3.)))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(1, activation='sigmoid'))
    sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False)
    model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [18]:
def do_normalize_multiple_dropout(epochs=100, dout = .2):
    dropout = dout
    numpy.random.seed(seed)
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('mlp', KerasClassifier(build_fn=create_dropout_multiple_dropout, epochs=epochs, batch_size=5, verbose=0)))
    pipeline = Pipeline(estimators)
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
    results = cross_val_score(pipeline, X_train, Y_train, cv=kfold)
    print("DropOut %.2f" % (dropout))
    print("Hidden: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100)) 

In [19]:
do_normalize_multiple_dropout(100, .3)
do_normalize_multiple_dropout(100, .4)
do_normalize_multiple_dropout(100, .5)


DropOut 0.30
Hidden: 50.00% (3.60%)
DropOut 0.40
Hidden: 51.00% (3.46%)
DropOut 0.50
Hidden: 49.52% (3.57%)


In [20]:
dropout = .2
def create_dropout_larger():
    model = Sequential() 
    model.add(layers.Dense(60, activation='relu', kernel_constraint=max_norm(3.), input_shape=(60, )))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(40, activation='relu', kernel_constraint=max_norm(3.)))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(20, activation='relu', kernel_constraint=max_norm(3.)))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(1, activation='sigmoid'))
    sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False)
    model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [21]:
def do_normalize_larger(epochs=100, dout = .2):
    dropout = dout
    numpy.random.seed(seed)
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('mlp', KerasClassifier(build_fn=create_dropout_larger, epochs=epochs, batch_size=5, verbose=0)))
    pipeline = Pipeline(estimators)
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
    results = cross_val_score(pipeline, X_train, Y_train, cv=kfold)
    print("DropOut %.2f" % (dropout))
    print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100)) 

In [22]:
do_normalize_larger(100, 0.3)

DropOut 0.30
Larger: 49.05% (3.47%)


In [23]:
dropout = 0.2
def create_dropout_mix():
    model = Sequential()    
    model.add(layers.Dropout(0.2, input_shape=(60, )))
    model.add(layers.Dense(60, activation='relu'))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(30, activation='relu'))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(1, activation='sigmoid'))
    sgd = SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=False)
    model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [24]:
def do_normalize_mix(epochs=100, dout = .2):
    dropout = dout
    numpy.random.seed(seed)
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('mlp', KerasClassifier(build_fn=create_dropout_mix, epochs=epochs, batch_size=5, verbose=0)))
    pipeline = Pipeline(estimators)
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
    results = cross_val_score(pipeline, X_train, Y_train, cv=kfold)
    print("DropOut %.2f" % (dropout))
    print("Mix: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100)) 

In [25]:
do_normalize_mix(100, .4)

DropOut 0.40
Mix: 87.38% (7.84%)


In [26]:
dropout = 0.2
max_weight = 4
def create_dropout_maxnorm():
    model = Sequential()    
    model.add(layers.Dropout(0.2, input_shape=(60, )))
    model.add(layers.Dense(60, activation='relu', kernel_constraint=max_norm(max_weight)))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(30, activation='relu', kernel_constraint=max_norm(max_weight)))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(1, activation='sigmoid', kernel_constraint=max_norm(max_weight)))
    sgd = SGD(lr=0.01, momentum=0.95, decay=0.0, nesterov=False)
    model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [27]:
def do_normalize_maxnorm(epochs=100, dout = .2, maxweigth=3):
    dropout = dout
    max_weight = maxweigth
    numpy.random.seed(seed)
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('mlp', KerasClassifier(build_fn=create_dropout_mix, epochs=epochs, batch_size=5, verbose=0)))
    pipeline = Pipeline(estimators)
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
    results = cross_val_score(pipeline, X_train, Y_train, cv=kfold)
    print("MaxNorm %.2f" % (max_weight))
    print("DropOut %.2f" % (dropout))
    print("MaxNorm: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100)) 

In [ ]:
do_normalize_maxnorm(100, .2, 3)
do_normalize_maxnorm(100, .2, 4)
do_normalize_maxnorm(100, .2, 5)

MaxNorm 3.00
DropOut 0.20
MaxNorm: 86.52% (6.00%)
MaxNorm 4.00
DropOut 0.20
MaxNorm: 84.09% (5.46%)
